In [ ]:
import synthcity
import sdv
import numpy as np
import os
import pandas as pd
import sdv.metadata.single_table as m
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.single_table import CTGANSynthesizer
from arfpy import arf
from synthcity.plugins.core.dataloader import GenericDataLoader
import random

In [ ]:
random.seed(031125)

In [ ]:
current_directory = os.getcwd()

In [ ]:
data_path = os.path.join(current_directory, "data")


dataset = "menobalance"
# Read the CSV file

if dataset == "menobalance":
   df = pd.read_csv(path + "menobalance_data.csv")
   df = df.iloc[:,[0,1,2,3,4,5,6]]
   current_directory = os.path.join(current_directory, "menobalance")
if dataset == "wisoncin":
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    df = pd.read_csv(url, header=None)
    df = df.iloc[:,[14,26,28,29]]
    df.columns = ["14","26","28","29"]
    current_directory = os.path.join(current_directory, "wisconsin")
if dataset == "cleveland":
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
    df = pd.read_csv(url, header=None)
    df = df.iloc[:,[3,7,4,0]]
    df.columns = ["3","7","4","0"]
    current_directory = os.path.join(current_directory, "cleveland")
if dataset == "simulated_gmcm": 
    simulated_path = "C:/Users/antoi/Documents/UNIBE/Synthetic data generation/results_paper/"
    df = pd.read_csv(simulated_path + "simulated_GMCM_data.csv")
    df = df.iloc[:,[1,2]]
    current_directory = os.path.join(current_directory, "simulated_gmcm")

In [ ]:

number_of_synthetic_datasets = 100
number_of_samples = 1000

Gaussian Copula

In [ ]:
gc_synthetizer_directory = os.path.join(current_directory, "GC_synthetizer")
os.makedirs(gc_synthetizer_directory, exist_ok=True)
i = 1
os.chdir(gc_synthetizer_directory)
synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(df)
synthetic_data = synthesizer.sample(num_rows=number_of_synthetic_datasets*number_of_samples)
for i in range (1,number_of_synthetic_datasets+1):
    synthetic_data[number_of_samples*(i-1):number_of_samples*i].to_csv(f"gc_samples_{i}.csv", index=False)  
    i = i + 1

CTGAN Synthetizer

In [ ]:
i = 1
ctgan_synthetizer_directory = os.path.join(current_directory, "CTGAN synthetizer")
os.makedirs(ctgan_synthetizer_directory, exist_ok=True)
os.chdir(ctgan_synthetizer_directory)
ctgan_synthesizer = CTGANSynthesizer(
        metadata, # required
        enforce_rounding=False,
        epochs=500,
        verbose=True
)
ctgan_synthesizer.fit(df)
for i in range (1,number_of_synthetic_datasets+1): 
    ctgan_samples = ctgan_synthesizer.sample(number_of_samples)
    ctgan_samples.to_csv(f"ctgan_samples_{i}.csv", index=False)  
    i = i + 1


AEF Synthetizer

In [ ]:
arf_synthetizer_directory = os.path.join(current_directory, "ARF")
os.makedirs(arf_synthetizer_directory, exist_ok=True)
os.chdir(arf_synthetizer_directory)
if dataset == "menobalance":
    columns_to_convert = ['Smoker','Diabetes']  # Specify the columns you want to convert
    df[columns_to_convert] = df[columns_to_convert].astype(object)
my_arf = arf.arf(x = pd.DataFrame(df))
my_arf.forde()
for i in range (1,number_of_synthetic_datasets+1): 
    samples_arf = my_arf.forge(n = number_of_samples )
    samples_arf.to_csv(f"arf_samples_{i}.csv", index=False)